In [464]:
import pandas as pd


In [465]:
df = pd.read_csv('movie_metadata.csv')

In [466]:
movie_data = df.copy()
movie_data.head(2)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0$,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0$,2007.0,5000.0,7.1,2.35,0


### Сколько пропущенных значений в Duration? (15)

In [467]:
movie_data['duration'].isna().value_counts()

False    5028
True       15
Name: duration, dtype: int64

### Замените пропущенные значения в колонке duration медианным значением этой колонки.
Какое среднее арифметическое по длительности фильма? Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно. (107.19)

In [468]:
movie_data['duration'] = movie_data['duration'].fillna(movie_data['duration'].median())
round(movie_data['duration'].mean(), 2)

107.19

### Создайте колонку movie_duration_category, которая содержала бы 3 категории в зависимости от длительности фильма:

Категория "1. < 90", если фильм длится менее 90 минут;
Категория "2. 90-120", если фильм длится от 90 минут до 2 часов (включительно);
Категория "3. > 120", если фильм длится больше 2 часов.
Постройте сводную таблицу для фильмов с годом выпуска после 2000 (включительно), которая отражала бы количество фильмов:

В строках таблицы - год;
В столбцах таблицы - категория длительности фильма ("< 90", "90-120", "> 120");
Год фильма должен быть представлен в формате YYYY
Сколько фильмов с длительностью от 90 минут до 2 часов было выпущено в 2008 году? (160)

In [469]:
def get_movie_duration(duration):
    if duration < 90:
        return '1. < 90'
    if 90 <= duration <= 120:
        return '2. 90-120'
    if duration > 120: 
        return '3. > 120'
    
movie_data['movie_duration_category'] = movie_data['duration'].apply(get_movie_duration)

piv_tab = pd.pivot_table(
    movie_data[movie_data['title_year'] >= 2000],
    values='movie_title',
    columns = 'movie_duration_category',
    index = 'title_year',
    aggfunc='count',
    fill_value=0
)
piv_tab

movie_duration_category,1. < 90,2. 90-120,3. > 120
title_year,,,
2000.0,25,112,34
2001.0,29,120,39
2002.0,36,146,27
2003.0,31,108,30
2004.0,30,142,42
2005.0,31,142,48
2006.0,40,146,53
2007.0,31,130,43
2008.0,29,160,36


### Колонка plot_keywords содержит ключевые слова, характеризующие сюжет фильма. На основе данных этой колонки создайте колонку movie_plot_category, которая содержала бы 4 категории в зависимости от ключевых слов в колонке:

Категория "love_and_death", если ключевые слова содержат и ключевое слово "love" и ключевое слово "death";
Категория "love", если ключевые слова содержат ключевое слово "love";
Категория "death", если ключевые слова содержат ключевое слово "death";
Категория "other", если ключевые слова не соответствуют перечисленным выше условиям.
Обратите внимание на разделитель ключевых слов. Подумайте о том как правильно определять наличие ключевого слова в списке ключевых слов.

В колонке imdb_score указан пользовательский рейтинг фильма. Постройте таблицу, которая отражала бы средний рейтинг фильмов в зависимости от принадлежности к категории movie_plot_category.

Какой средний рейтинг у фильмов категории "love"? Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно. (6.53)

In [470]:
def get_movie_category(text):
    if pd.isna(text):
        return None
    else:
        
        keywords_list = text.split('|')
        if 'love' in keywords_list and 'death' in keywords_list:
            return 'love_and_death'
        elif 'love' in keywords_list:
            return 'love'
        elif 'death' in keywords_list:
            return 'death'
        else:
            return 'other'

movie_data['movie_plot_category'] = movie_data['plot_keywords'].apply(get_movie_category)
avg_rating = round(movie_data.groupby(['movie_plot_category'], as_index=False)['imdb_score'].mean(), 2)
avg_rating

,movie_plot_category,imdb_score
0,death,6.51
1,love,6.53
2,love_and_death,6.68
3,other,6.45


### Колонка budget - бюджет фильма. Какой медианный бюджет всех представленных фильмов? Ответ дайте целым числом.
15000000

In [471]:
movie_data['budget'] = movie_data['budget'].str.replace('$', '').astype(float)
int(movie_data['budget'].median())

<ipython-input-471-a8d998bc6b21>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  movie_data['budget'] = movie_data['budget'].str.replace('$', '').astype(float)


15000000